In [14]:
import nltk; nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/UniversalEmb

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/UniversalEmb


In [13]:
!git clone https://github.com/jasonwei20/eda_nlp

Cloning into 'eda_nlp'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 379 (delta 18), reused 1 (delta 0), pack-reused 349
Receiving objects: 100% (379/379), 20.41 MiB | 13.57 MiB/s, done.
Resolving deltas: 100% (181/181), done.


In [49]:
import pandas as pd
n_examples = 4000

labels = [i.split(' ', 1 )[0].split(':')[1] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_examples]
texts = [ i.split(' ', 1 )[1][:-1] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_examples]
train  = pd.DataFrame({'texts':texts, 'labels': labels})

labels_t = [i.split(' ', 1)[0].split(':')[1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
texts_t = [ i.split(' ', 1)[1][:-1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
test  = pd.DataFrame({'texts':texts_t, 'labels': labels_t})
test.shape

(500, 2)

In [0]:
f = open('trec_for_eda.txt', 'w')
for t, l in zip(texts,labels):
  f.write(l+'\t'+t+'\n')
f.close()

In [51]:
!python eda_nlp/code/augment.py --input=trec_for_eda.txt --output=trec_for_eda_augmented.txt --num_aug=4 --alpha=0.1

generated augmented sentences with eda for trec_for_eda.txt to trec_for_eda_augmented.txt with num_aug=4


In [52]:
g = open('trec_for_eda_augmented.txt', 'r').readlines()
train  = pd.DataFrame({'texts':[i[:-1].split('\t')[1]for i in g], 'labels': [i[:-1].split('\t')[0]for i in g]})
train.head()

,texts,labels
0,then did serfdom develop in and how leave russia,manner
1,then did serfdom develop in and how leave russia,manner
2,how did serfdom develop in and then leave russ...,manner
3,how did serfdom develop in and then leave sovi...,manner
4,how did serfdom develop in and then leave russia,manner


In [0]:
tr = {i:j for j, i in enumerate(train.labels.unique().tolist())}
def t(f):
  return tr[f]
train['lb'] = train.labels.apply(t)
test['lb'] = test.labels.apply(t)                  

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

In [55]:
tf = TfidfVectorizer(lowercase = False, ngram_range = (1,3), max_df =0.98, min_df = 3)
tf_res = tf.fit_transform(train.texts)
test_transform = tf.transform(test.texts)
tf_res.shape

(20000, 44313)

In [0]:
import numpy as np
from sklearn.metrics import classification_report
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import time

In [0]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

class WordData(Dataset):
    
    def __init__(self, x_data, y_data):
        
        super().__init__()
        
        self.x_data = x_data
        self.y_data = torch.LongTensor([int(i) for i in y_data])
        
    
    def __len__(self):
        
        return self.x_data.shape[0]
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx].toarray()
        x = torch.FloatTensor(x)
        y = self.y_data[idx]
        
        return x, y


dataset_train = WordData(tf_res, train.lb)
dataset_test = WordData(test_transform, test.lb)

batch_size = 256

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,  shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [0]:
from math import sqrt
class MLP(torch.nn.Module):
    
    def __init__(self, in_features, inner_features, inner_features2, out_features):
        
        # Главная задача этого метода - возможность использования в классе потомке, методов класса-родителя
        # Вызываем __init__ родителя - torch.nn.Module
        super().__init__()
        
        # определяем слои и все что захотим сохранять/использовать
        self.linear_1 = torch.nn.Linear(in_features=in_features, out_features=inner_features)
        gl = sqrt(6./(in_features+inner_features))
        self.linear_1.weight.data.uniform_(-gl, gl)
        self.non_linear_function = torch.nn.ReLU()
        self.drop_out = torch.nn.Dropout(0.18)
        self.linear_2 = torch.nn.Linear(in_features=inner_features, out_features=inner_features2)
        gl1 = sqrt(6./(inner_features+inner_features2))
        self.linear_2.weight.data.uniform_(-gl1, gl1)
        self.non_linear_function2 = torch.nn.ReLU()
        self.linear_3 = torch.nn.Linear(in_features=inner_features2, out_features=out_features)
        gl2 = sqrt(6./(inner_features2+out_features))
        self.linear_3.weight.data.uniform_(-gl2, gl2)
  
    def forward(self, x):
        
        # наша привычная схема
        x = self.linear_1(x)
        x = self.non_linear_function(x)
        x = self.drop_out(x)
        x = self.linear_2(x)
        x = self.non_linear_function2(x)
        x = self.linear_3(x)
        
        return x
    

In [0]:
model = MLP(in_features=tf_res.shape[1], inner_features=524, inner_features2=256, out_features=len(set(train.lb)))

In [0]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0005,  betas=(0.9, 0.999) )
# aka loss function
criterion = torch.nn.CrossEntropyLoss()

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [62]:
from sklearn.metrics import f1_score
epochs = 35
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        pred = model(x).squeeze(1)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y in validation_loader:     
        
        x = x.to(device)
        y = y.to(device)
       

        with torch.no_grad():
            pred = model(x).squeeze(1)

            pred = pred.cpu()
            y = y.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.7f}, test - {:.7f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.4f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Epoch 2:   0%|          | 0/20000 [00:00<?, ?it/s]


Losses: train - 2.9610558, test - 2.5081531
F1 test - 0.4500


Epoch 3:   0%|          | 0/20000 [00:00<?, ?it/s]


Losses: train - 0.8413674, test - 1.7694330
F1 test - 0.6180


Epoch 4:   0%|          | 0/20000 [00:00<?, ?it/s]

...


Losses: train - 0.0085168, test - 1.5105447
F1 test - 0.6660


Epoch 9:   0%|          | 0/20000 [00:00<?, ?it/s]


Losses: train - 0.0062314, test - 1.4915742
F1 test - 0.6640


Epoch 9: 100%|██████████| 20000/20000 [00:54<00:00, 367.75it/s, train_loss=0.0243]



Losses: train - 0.0047136, test - 1.4949136
F1 test - 0.6540
Early stopping


In [0]:
for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)